In [1]:
import pandas as pd 
import numpy as np
from sklearn import tree
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import os
import array
import warnings
from sklearn.metrics import ( 
    accuracy_score, 
    confusion_matrix, 
    ConfusionMatrixDisplay, 
    f1_score, 
    classification_report, 
)
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
os.system('cls')

# import pandas as pd 
from mlxtend.frequent_patterns import apriori, association_rules

# import numpy as np
import warnings
warnings.filterwarnings('ignore')

import random

In [2]:
df = pd.read_csv("Training.csv")
test=""
symptom=['itching','skin_rash','nodal_skin_eruptions']
symptom=list(set(symptom))
disease=set(df['prognosis'])
disease=list(disease)
disease.sort()
feature_cols =list(df.columns)
feature_cols.pop()

# -------------------------
X = df[feature_cols] 
y = list(df['prognosis'])
y = pd.get_dummies(y,drop_first=False)
y_col=y.columns.to_list()
# -------------------------

In [3]:
def patient():
    global test
    patient_data = pd.read_csv("a3_binary.csv")
    # patient_data=patient_data.drop("prognosis", axis=1)
    # x=patient_data.iloc[0]
    # print(tuple(patient_data))
    # print(x[0])
    test=patient_data
    test=[patient_data.iloc[0]]
# patient()

In [4]:
decisionTree_final=[]
RandomForest_final=[]
svm_final=[]

multi_output_tree=""
multi_output_svm=""
multi_output_Forest=""

In [5]:
def disease_output(e,f):
    global y_col
    
    for i in range(len(e)):
        temp=[]
        for j in range(len(e[0])):
            if(e[i][j]==True):
                temp.append(y_col[j])
        f.append(temp)


In [6]:
def SVM():
    global multi_output_svm
    global X,y
    clf = svm.SVC(kernel='linear')
    multi_output_svm=MultiOutputClassifier(clf)
    multi_output_svm.fit(X, y)
    
def SVM_outpt():
    global test
    global multi_output_svm
    global svm_final
    svm_final=[]
    predict= multi_output_svm.predict(test)
    disease_output(predict,svm_final)

In [7]:
def decisionTree():
    global multi_output_tree
    global X,y
    dtree = DecisionTreeClassifier() 
    multi_output_tree=MultiOutputClassifier(dtree)
    multi_output_tree.fit(X,y) 

def decisionTree_Output():
    global test
    global multi_output_tree
    global decisionTree_final
    decisionTree_final=[]
    predict = multi_output_tree.predict(test)
    disease_output(predict,decisionTree_final)

In [8]:
def RandomForest():
    global multi_output_Forest
    global X,y

    # ---------------------------------------------------   
    rf = RandomForestClassifier()
    # multi_output_Forest=rf
    multi_output_Forest=MultiOutputClassifier(rf)
    multi_output_Forest.fit(X,y) 
    # ---------------------------------------------------   
    # rf = RandomForestClassifier(n_estimators=74,max_depth=49)
    # multi_output_Forest=MultiOutputClassifier(rf)
    # multi_output_Forest.fit(X,y) 


    # ---------------------------------------------------   
    # param_dist = {'n_estimators': randint(50,200), 'max_depth': randint(10,100)}
    # rf = RandomForestClassifier()
    # rand_search = RandomizedSearchCV(rf, param_distributions = param_dist, n_iter=5, cv=5)
    # rand_search.fit(X, y)
    # print(rand_search.best_estimator_)
    # multi_output_Forest=MultiOutputClassifier(rand_search.best_estimator_)
    # multi_output_Forest.fit(X,y)
    # # print(rand_search.best_estimator_)

# RandomForest()

In [9]:
def RandomForest_Output():
    global test
    global multi_output_Forest
    global RandomForest_final
    RandomForest_final=[]
    predict = multi_output_Forest.predict(test)
    disease_output(predict,RandomForest_final)

In [10]:
decisionTree()
RandomForest()
SVM()

# decisionTree_Output()
# RandomForest_Output()
# SVM_outpt()

In [11]:
import json
with open('test_pair.json','r') as openfile:
    pair=json.load(openfile)
patient()
decisionTree_Output()
RandomForest_Output()
SVM_outpt()

In [12]:
def Accuracy():
    r=0
    d=0
    s=0
    for i in range(len(RandomForest_final)):
        if (len(decisionTree_final[i])>=len(pair[i])):
            d=d+1
        if (len(RandomForest_final[i])>=len(pair[i])):
            r=r+1
        if (len(svm_final[i])>=len(pair[i])):
            s=s+1
    r=r/len(RandomForest_final)*100
    d=d/len(RandomForest_final)*100
    s=s/len(RandomForest_final)*100
    print(len(RandomForest_final))
    print('r=',r,'d=',d,'s=',s)
Accuracy()

1
r= 100.0 d= 100.0 s= 100.0


In [18]:
def Apriori():
    global rules
    df=pd.read_csv('Apriori copy.csv')
    z=pd.concat([df['Disease1'],df['Disease2']])
    z=list(set(z))
    z.sort()
    p=np.zeros((len(df),len(z)),dtype=int)
    p=pd.DataFrame(p)
    p=p.set_axis(z,axis=1) 
    for i in range(len(df)):
        for x in df.iloc[i]:
            p[x].iloc[i]=1

    frequent_itemsets = apriori(p, min_support=0.01, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0)
    print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values(by=['confidence'],ascending=0))
Apriori()
# print(rules)

          antecedents        consequents   support  confidence       lift
29                (h)  (Hyperthyroidism)  0.044872    1.000000  12.000000
27               (gg)  (Hyperthyroidism)  0.038462    1.000000  12.000000
13               (kk)           (Dengue)  0.162393    1.000000   6.157895
12           (Dengue)               (kk)  0.162393    1.000000   6.157895
11         (Dementia)     (Hypertension)  0.010684    0.833333   5.000000
23   (Osteoarthritis)     (Hypertension)  0.012821    0.600000   3.600000
28  (Hyperthyroidism)                (h)  0.044872    0.538462  12.000000
26  (Hyperthyroidism)               (gg)  0.038462    0.461538  12.000000
1         (Arthritis)       (Depression)  0.012821    0.193548   1.484929
16    (Heart Disease)       (Depression)  0.012821    0.176471   1.353905
8            (Cancer)     (Hypertension)  0.017094    0.166667   1.000000
6            (Cancer)       (Depression)  0.014957    0.145833   1.118852
19       (Depression)     (Hypertensio

In [19]:
def predict_apriori(intersection):
    global rules
    
    apriori_final=[]
    for x in intersection:
        # print(x)
        for i in range(len(rules)):
            if(x in rules.iloc[i]['antecedents']):
                apriori_final.append(rules.iloc[i])
    apriori_final=pd.DataFrame(apriori_final)
    # print(apriori_final)        
    print(apriori_final[['antecedents', 'consequents', 'support', 'confidence', 'lift','jaccard']].sort_values(by=['confidence'],ascending=0))
    return apriori_final[['antecedents', 'consequents', 'support', 'confidence', 'lift','jaccard']].sort_values(by=['confidence'],ascending=0)
# predict_apriori()

In [20]:
# a=['Dengue','Hyperthyroidism','Dengue','Hyperthyroidism','Malaria','hh','Malaria']
# int=['Dengue','Hyperthyroidism']

In [21]:
def Disease_percentage():
    global intersection
    print('RandomForest_final',RandomForest_final)
    print('decisionTree_final',decisionTree_final)
    print('svm_final         ',svm_final)
    print('---------------------------------------')
    union=[]
    union=RandomForest_final[0].copy()
    union.extend(decisionTree_final[0])
    union.extend(svm_final[0])
    # union_set=list(set(union))
    intersection=list(set(RandomForest_final[0]).intersection(set(decisionTree_final[0]) , set(svm_final[0])))
    #for testing
    # union=a
    # intersection=int

    # print('intersection',intersection)
    u_min_n=[x for x in union if x not in intersection]
    u_min_n_set=list(set(u_min_n))
    final=[[x,100] for x in intersection]
    # print('u_min_n     ',u_min_n)
    # print('final       ',final)
    # print('u_min_n_set ',u_min_n_set)
    for x in u_min_n_set:
        final.append([x,u_min_n.count(x)/(3*len(u_min_n_set))*100])
    final=pd.DataFrame(final)
    final.columns=['Disease','%']
    print(final.sort_values(by='%',ascending=0))
    print('---------------------------------------')
    apriori_final=predict_apriori(intersection)
    print('---------------------------------------')
    final_apriori=[]
    for i in range(len(apriori_final)):
        print((apriori_final.iloc[i]['consequents']))
        final_apriori.append([apriori_final.iloc[i]['consequents'], apriori_final.iloc[i]['confidence']*100])
    final_apriori=pd.DataFrame(final_apriori)
    final_apriori.columns=['Disease','%']
    print(final_apriori)
Disease_percentage()

RandomForest_final [['Dengue', 'Hyperthyroidism']]
decisionTree_final [['Dengue', 'Hyperthyroidism', 'Malaria']]
svm_final          [['Dengue', 'Hyperthyroidism']]
---------------------------------------
           Disease           %
0           Dengue  100.000000
1  Hyperthyroidism  100.000000
2          Malaria   33.333333
---------------------------------------
          antecedents consequents   support  confidence       lift   jaccard
12           (Dengue)        (kk)  0.162393    1.000000   6.157895  1.000000
28  (Hyperthyroidism)         (h)  0.044872    0.538462  12.000000  0.538462
26  (Hyperthyroidism)        (gg)  0.038462    0.461538  12.000000  0.461538
---------------------------------------
frozenset({'kk'})
frozenset({'h'})
frozenset({'gg'})
  Disease           %
0    (kk)  100.000000
1     (h)   53.846154
2    (gg)   46.153846
